In [1]:
%reload_ext autoreload
%autoreload 2

"""
mc4を掃除して､fasttextでgood/badを分類して､クラスタリングして記録するサンプルコード
annotationもこのnotebookで行います

"""
from datasets import load_dataset


#mc4の読み込み
mc4_dataset = load_dataset('mc4', 'ja',split='train', streaming=True)

"""
#oscarなども読み込める
#ignore_verifications=Trueをつけないとエラーとなる
oscar_dataset = load_dataset('oscar', 'unshuffled_original_ja', 
                       split='train', 
                       ignore_verifications=True,
                       streaming=True)

"""

dataset=mc4_dataset

/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniconda3/envs/crawl/lib/python3.12/site-packages/datasets/load.py:1454: FutureWarning: The repository for mc4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mc4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [2]:
from src.cleaner.auto_cleaner import clean_text
from src.classifier.DatasetAnnotator import DatasetAnnotator

In [3]:
annotator=DatasetAnnotator(dataset,clean_func=clean_text,n_preload=50000)

In [12]:
#annotations
annotator.ask_annotations()


begin annotations
Enter: annnotate as "bad"
something: annotate as "good"
q: quit



In [15]:
#fasttextで訓練
annotator.train_fasttext(autotuneDuration=120,wordNgrams=2)

Outputting annotations
Annotations saved to annotations/text_labels
Training fasttext model


Progress: 100.0% Trials:   98 Best score:  0.762500 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  22568
Number of labels: 2
Progress: 100.0% words/sec/thread: 4135559 lr: -0.000069 avg.loss:  0.163856 ETA:   0h 0m 0s

Model saved to annotations/text_labels/model.bin
Testing model
(81, 0.8148148148148148, 0.8148148148148148)


Progress: 100.0% words/sec/thread: 4135109 lr:  0.000000 avg.loss:  0.163856 ETA:   0h 0m 0s


In [7]:
#テキストの教師なしクラスタリング

#modelをannotations/text_labelsに保存しておく
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ja.300.bin.gz
#!gzip -d cc.ja.300.bin.gz

In [16]:
from src.classifier.NounClustering import NounClustering
clf=NounClustering()

#clf.train_wiki(n_samples=10**8) #wikipediaのタイトルで教師なしクラスタリングの学習

Loading fasttext model
kmeans loaded: annotations/text_labels/kmeans.pkl


In [18]:
#モデルの挙動確認
#カテゴリの分類がかなりいまいち｡
for i in range(100):
    text=annotator.get_text_by_id(i)
    text_=text.replace("\n","")
    if text=="":continue
    is_noise=(annotator.predict(text))
    genre=clf.predict(text)
    print(is_noise,genre,text_[:300])

0 37 生八つ橋のタグまとめ|エキサイトブログ生八つ橋のタグまとめ「生八つ橋」のタグがついている新着記事と人気記事をまとめました。エキサイトブログには生八つ橋に関連するブログ(日記、記録、写真、レビュー、噂、まとめ)がたくさん投稿されています。「生八つ橋」タグの記事(4)生八つ橋いろいろ京都旅行のお土産(我が家用)に色々な生八つ橋を買ってきました。我が家はみんな八つ橋ファンなのです～。地元の方達や京都を案内してくれたYさんは「八つ橋なんてもう何年も食べたことないわ～～～」と仰っていましたが、いやいや、美味しいですよ～～!私は大好きです!まぁ確かに私は東京出身ですが、雷おこしや人形焼きは食べませんからね
1 97 廃棄物をチップ/ペレットに-Gneussフィルム、繊維、更には市場回収品であるPETボトルフレークなどのリサイクル可能な廃棄物は、高水準の純度で高品質ペレット/チップに加工し、生産押出機でバージン材料と混ぜることができます。繊維、フィルムやダンゴなどのリサイクル可能な廃棄物は、シュレッダーに入れられ、金属セパレーター付コンベアでアジエーターを搭載した容器に投入されます。そこから、材料はオーガや詰め込みフィーダーによって押出機に運ばれます。あるいは、サイズを圧縮し塊となったリサイクル可能な廃棄物は、MRS押出機に直接入れることができます。サイズ圧縮と押し出しの処理工程を切り離すことの長所は、材料
0 37 Reimsくもり中華の後のランデブーは公証人。ニンニクの臭いを気にしつつ。到着前に、更にブレスケアを仕込み。今まで、自分の持ち物のように家を紹介しておりましたが、実はまだ正式な契約は済ませておらず、住宅メーカーの持ち物でした。見栄っ張りでスミマセン・・・晴れて4月2日、契約書にサインをし、自分たちの持ち家になったわけです。パチパチパチパチ^^公証人役場の前の桜も満開で、祝福してくれているようです。まぁ、色々ゴタゴタしていて、当初1月にサインする予定が、来月、来月、来月、、、、、未定・・・となって、これも先週急に決まった日にち。紙面ではなく、画面を見ながら、書類(自宅に事前に送られてきているもの
1 97 サーモス真空断熱ポットコーヒーメーカー【タイマーで前日予約が可能】ECH-1...の見た目の良さと、渋くなりにくい良さ,2014/10/12定年した父に、

In [51]:
n_split=10**2
import os

corpus_dir="corpus/test"
if not os.path.exists(corpus_dir):
    os.makedirs(corpus_dir)

cnt=0
record_id=-1
for record in dataset:
    record_id+=1
    text=record['text']

    #テキストクリーン
    text=clean_text(text)
    if text=="":
        continue

    #記事の判定
    is_noise=annotator.predict(text)
    if is_noise==1:
        continue

    #ジャンルの判定
    genre=clf.predict(text)
    cnt+=1

    d={
        "id":record_id,
        "cat":genre,
        "text":text,
    }
    file_name=f"{corpus_dir}/{cnt//n_split}.txt"
    with open(file_name, "a") as f:
        f.write(str(d)+"\n")

    if cnt>1000:
        break

In [52]:
#圧縮率
cnt/record_id

0.1817686580715453